# Main parallel computation, test time usage 

In [117]:
import numpy as np 
import math 


from multiprocessing import Pool

import time

import parameters_module as pr 
import sys 

# from numba import jit 
# from numba.extending import overload        

# metropolis sampling and integral approximation 
%run functions_module.ipynb 
%run auxiliaryFunctions_noMath_module.ipynb 
%run gradientDescent_module.ipynb 

# Acceleration suggestions 

# User definable variables 

**Input:** Initial guess for parameters (don't set 0, otherwise Division By Zero -error occurs): 

In [118]:
parametersList = [0.2, 0.2] 

**Input:** Number of parallel configuration subspaces (parallel configuration spaces)

In [119]:
M =10

**Input**: Filenames for the Python-compatible formulae of probability function and local energy (Supposed to be in the default folder "Formulae_PythonCompatible"  )

In [120]:
probabilityExprPath = "probability_Helium"


localEnergyExprPath = "localEnergy_Helium"

**Input**: Samples/sampling settings in Metropolis sampling

- Tips: 
    - Coordinate value range chosen so that it is close to the known radius of the atom.
- Notes: 
    - Unit of coordinates is hydrogen radius (Hatree atomic units)

In [121]:
coordinateValueRange= (-3, 3)
numberOfParticles = 2
numberOfConfig = 20  
numberOfIterations = 60             # for thermalisation            

**Input**: Tolerance on energy variance

In [122]:
toleranceOnEnergyVariance = 0.001

**Input**: if you want to optimize parameters by hand

- if optimizing by hand, there will be no checkpoints

In [123]:
wantParamsOptimizedByHand = False

# Preparing local energy functions and probability functions 

## Load in probability and local energy expressions 

In [124]:
# default folder to the Python-compatible formulae 
defaultFolderPathForFormulae = sys.path[0]+ "/"+ "Formulae_PythonCompatible/"    

In [125]:
# global variables: used in probability and local energy functions definition

probabilityExpr = readFileToString(defaultFolderPathForFormulae + probabilityExprPath)  

localEnergyExpr = readFileToString(defaultFolderPathForFormulae + localEnergyExprPath)

In [126]:
probabilityExpr

'math.exp(-2*A3*math.sqrt(x1**2 + y1**2 + z1**2) - 2*A3*math.sqrt(x2**2 + y2**2 + z2**2) -     (2*A2*(1 - math.exp          (-math.sqrt(abs(x1 - x2)**2 + abs(y1 - y2)**2 + abs(z1 - z2)**2)/            math.sqrt(A2))))/     math.sqrt(abs(x1 - x2)**2 + abs(y1 - y2)**2 + abs(z1 - z2)**2))'

In [127]:
localEnergyExpr[:50] 

'math.exp(A3*math.sqrt(x1**2 + y1**2 + z1**2) + A3*'

## Find variables in the expressions

In [128]:
# find coordinates and parameters
coordinates, parameterStringList  = findVariables(localEnergyExpr)

# regrouping coordinates 
coordinates = regroupToCoordinateTriple_findVariables(coordinates)


# for functions usage, define list of ordered variables and parameters
mapping =  coordinates + parameterStringList  # global variable: used in probability and local energy functions definition


In [129]:
coordinates, parameterStringList, mapping 

(['x1', 'y1', 'z1', 'x2', 'y2', 'z2'],
 ['A2', 'A3'],
 ['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'A2', 'A3'])

### Error check: check number of initial guess is same as number of parameters found

In [130]:
check_numberOfIni_VS_numberOfFound_parameters(parametersList, parameterStringList) # raises exception if error occurs, program stopped

## Define probability and local energy functions

In [131]:

def probabilityFunc(R: np.ndarray, parameters: list, expr: str = probabilityExpr, mapping: list = mapping) -> float: 
    
    '''
    psi^2. Probability function. Dependent on configuration and parameters. 
    
    Parameters
    ----------
        R: ordered current configuration e.g. 
            array([[-0.5583685 , -0.04608995,  0.15500853],
                 [ 0.66255653,  0.66301583, -0.85159876]])
             each row representing one single particle's position
        
        parameters: ordered list of parameters as numbers 
        
        expr: formula string. Default to the global variable 
        
        mapping: ordered coordinates and parameters in a list: 
            ['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3', 'A1', 'A2']. Default to the global variable 
        
    Return
    ------
        Probability function evaluated at certain configuration and parameters' values
    '''
    
    
    
    
    R_flattened = R.flatten() # make current configuration to vector form 
    R_and_parameters = np.append(R_flattened, parameters) # ordered coordinates, and parameters at the tail of the vector 
    
    
    if len(R_and_parameters) != len(mapping): 
        raise Exception("R and parameters not same length as that of mapping!")
    
    
    # mapping dictionary for evaluation of the expression
    localDict = dict(zip(mapping, R_and_parameters))

    
    
    return eval(expr, {'math': math}, localDict)
    
    
    
    


def localEnergyFunc(R: np.ndarray, parameters: list, expr: str = localEnergyExpr, mapping: list = mapping ) -> float: 
    '''
    
    H*psi/psi. Local energy.  Dependent on configuration and parameters. 
    
    Parameters
    ----------
        R: current configuration
        
        parameters: list of parameters as numbers 
        
        expr: formula string 
        
        mapping: ordered coordinates and parameters in a list: 
            ['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3', 'A1', 'A2']
            
    Return
    ------
        Local energy evaluated at certain configuration and parameters' values
    
    '''
    
    
    R_flattened = R.flatten() # make current configuration to vector form 
    R_and_parameters = np.append(R_flattened, parameters) # ordered coordinates, and parameters at the tail of the vector 
    
    
    if len(R_and_parameters) != len(mapping): 
        raise Exception("R and parameters not same length as that of mapping!")
    
    
    # mapping dictionary for evaluation of the expression
    localDict = dict(zip(mapping, R_and_parameters))

    
    
    
    
    return eval(expr, {'math': math}, localDict)
    
    
    

# Metropolis sampling and local energy estimations 

In [132]:
parametersHistoryDict = {}

In [133]:
energyDict = {'Energy': [], 
             'Variance': [], 
             'std error': [], 
             }  # to append list of 

## Configuration subspaces: getting samples for energy

In [134]:

# to save list of average energies from each configuration subspace 
energyInConfigSpaceList = []

# create variables with no specific meaning  for parallelization 
idx_list = list(range(M))






# define function for managing and perform configuration subspace computations 
def configurationSubspaceFunction(i): 
    
    
    start = time.time()
    # generating initial Metropolis samples and Thermalisation
    configSubspace_Metropolis = metropolisSamplingFunction( coordinateValueRange=  coordinateValueRange, 
                                                           numberOfParticles = numberOfParticles,  
                                                           numberOfConfig = numberOfConfig, 
                                                           probabilityFunction=probabilityFunc, 
                                                           numberOfIterations = numberOfIterations, 
                                                           params = parametersList)
    end = time.time()
    # print('Time used in thermalisation', end- start )
    
    energyInConfigSubspaceList = []           # for saving the energy values in this subspace 
    
    
    start = time.time()
    # energy computation for each Metropolis suggestion
    # for each configuration 
    for configIndex in range(configSubspace_Metropolis.shape[1]): 
        # current configuration in format: 
        #  array([[-0.5583685 , -0.04608995,  0.15500853],
       # [ 0.66255653,  0.66301583, -0.85159876]])
        # each row representing one particle's position  
        currentConfig = configSubspace_Metropolis[:, configIndex, :]   
        
        # for each particle's displacement, compute energy 
        for nthParticleIndex in range(len(currentConfig)): 
            # making Metropolis-suggestion on single particle
            updatedConfig = metropolisStepSuggestion(configuration= currentConfig, 
                                                    nthParticle= nthParticleIndex, 
                                                    probabilityFunction=probabilityFunc, 
                                                    params=parametersList)
            
            # update configuration subspace 
            configSubspace_Metropolis[:, configIndex, :] = updatedConfig
            
            
            # compute energy 
            energyInConfigSubspace = monteCarloIntegrationFunction(samples= configSubspace_Metropolis, 
                                                                     localEnergyFunction=localEnergyFunc, 
                                                                     params=parametersList)
            
            # append to the list 
            energyInConfigSubspaceList.append(energyInConfigSubspace)
    end = time.time()
    # print('Time used in computing list of energies in each Metropolis suggestion', end- start)
    # print(f'{i}.loop completed')
    
    
    # compute mean energy of this subspace and return it 
    return np.mean(energyInConfigSubspaceList)
    

        
    

# Run the program 

In [135]:

count = 0 # counter for how many complete loops 
while True: 
    print(f'Current optimization loop: {count}')

    # concurrent computing: get mean energies from each independent configuration subspaces
    print('In parallel computation, configuration subspace...')
    with Pool() as pool:
        start = time.time()
        for result in pool.map(configurationSubspaceFunction, idx_list  ): 
            energyInConfigSpaceList.append(result)
        end = time.time()
        print('Time used in parallel computation', end - start)

    ## compute energy approximation 

    # compute mean 
    energyMean =np.mean(energyInConfigSpaceList)
    


    print('Saving results...')
    # compute variance and std error of mean, if there are more than 1 samples 
    if len(energyInConfigSpaceList)> 1:
        # sample variance (N-1)
        energyVar =np.var(energyInConfigSpaceList, ddof= 1)

        # std error of mean 
        energyError = energyVar/len(energyInConfigSpaceList)


        # printing the values 
        print('Mean:', energyMean)
        print('Variance:', energyError)
        print('Std error of mean:', energyError)

        # saving values 
        energyDict['Energy'].append(energyMean)
        energyDict['Variance'].append(energyVar)
        energyDict['std error'].append(energyError)
    else: 
        print('Variance and std error of mean cannot be calculated: There is only 1 sample')
        print()
        print(energyMean)

        # saving values 
        energyDict['Energy'].append(energyMean)
        energyDict['Variance'].append(0)
        energyDict['std error'].append(0)
        
    
    
    # emptying energy list 
    energyInConfigSpaceList = []




    # Optimization of parameters 
    print('Optimizing...')

    start = time.time()
    # generate new Metropolis samples
    metropolisSamples_Opt = metropolisSamplingFunction( coordinateValueRange=  coordinateValueRange, 
                                                               numberOfParticles = numberOfParticles,  
                                                               numberOfConfig = numberOfConfig, 
                                                               probabilityFunction=probabilityFunc, 
                                                               numberOfIterations = numberOfIterations, 
                                                               params = parametersList)
    end = time.time() 
    print('Time used in generating new thermalised Metropolis samples for optimization', end- start )


    # break the loop: if user optimize the parameters by hand 
    if wantParamsOptimizedByHand == True: 
        break 

    start = time.time()
    # optimizing 
    parametersList = gradientDescentFunction(monteCarloIntegrationFunc=monteCarloIntegrationFunction, 
                                            function=localEnergyFunc, 
                                            learning_rate= 0.001, 
                                            samples= metropolisSamples_Opt, 
                                            parameters=parametersList, 
                                            tolerance = 0.001, 
                                            maximumSteps = 10, 
                                            showEvolution = False)
    end = time.time() 
    print("Time used in parameters' optimization", end - start)
    
    # append optimized parameters into history record
    parametersHistoryDict[count] = parametersList
    
    
    
    print('Checkpoint saving...')
    print()
    # checkpoint: save parameters and energy estimates to a file 
    saveParamsAndEnergyEstimates(parametersDict = parametersHistoryDict, 
                                 energyDict = energyDict , 
                                 parametersFilename = 'parameters_history.json', 
                                 energyFilename = 'energyEstimates_history.json')
    
    
    count +=1
   
    # Break the loop: when energy variance is close to tolerance defined by user 
    if count >1: 
        if energyDict['Variance'][-1] <= toleranceOnEnergyVariance: 
            break 

        



Current optimization loop: 0
In parallel computation, configuration subspace...
Time used in parallel computation 59.73589825630188
Saving results...
Mean: -0.976873077868127
Variance: 0.0022428334982247168
Std error of mean: 0.0022428334982247168
Optimizing...

Time used in generating new thermalised Metropolis samples for optimization 0.5187780857086182
Time used in parameters' optimization 31.24274516105652
Checkpoint saving...
Current optimization loop: 1
In parallel computation, configuration subspace...
Time used in parallel computation 60.69162154197693
Saving results...
Mean: -1.1127319384656587
Variance: 0.002425626313634425
Std error of mean: 0.002425626313634425
Optimizing...

Time used in generating new thermalised Metropolis samples for optimization 0.5073537826538086
Time used in parameters' optimization 29.11047339439392
Checkpoint saving...
Current optimization loop: 2
In parallel computation, configuration subspace...
Time used in parallel computation 67.54992985725403

KeyboardInterrupt: 